In [1]:
import pandas as pd
import re,sys,os

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import keras
import gensim

from time import time  # To time our operations
from collections import defaultdict  # For word frequency

# fix random seed for reproducibility
np.random.seed(7)
os.chdir("E:/pythontraining/completeimarticus_tutorial/DL/Rnn/")

C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
revDF=pd.read_csv('zomato-bangalore-restaurants/zomato.csv')

In [3]:
revDF.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [4]:
cusinedata=[]
for e in revDF['dish_liked'].values:
    if pd.isna(e) != True:
        for t in e.split(','):
            cusinedata.append(t.lower().strip())
for e in revDF['cuisines'].values:
    if pd.isna(e) != True:
        for t in e.split(','):
            cusinedata.append(t.lower().strip())

In [5]:
cusinedata=list(set(cusinedata))

In [6]:
len(cusinedata)

2885

In [7]:
cusinedata[:3]

['chicken chowmein', 'caramel ice cream', 'angara kebab']

In [8]:
[f for f in cusinedata if 'chicken puff' in f]

['chicken puff']

In [9]:
revDF['dish_liked'].values[1]

'Momos, Lunch Buffet, Chocolate Nirvana, Thai Green Curry, Paneer Tikka, Dum Biryani, Chicken Biryani'

In [10]:
def cln(x):
    x=re.sub('\\n','',x)
    x= re.sub('[^0-9 a-z A-Z \s\.,\\/]','',x)
    x=x.strip()
    x=re.sub("x83x83x82x82x83x82x82x92",'',x)
    return(x)

revDF.reviews_list[0].split('RATED')[2].split('Rated')[0]
clRev=[]

for rev in revDF.reviews_list:
    rev1=''
    for each in rev.split('RATED'):
        rev1 = rev1 + ',' + cln(each)
    clRev.append(rev1)
    
revDF['cl_review']=clRev

def ratingExt(x):
    try:
        r=float(x.split("/")[0])/int(x.split("/")[1])
    except:
        r=np.nan
    return r
        
revDF['ratingPer']=revDF['rate'].apply(lambda x : ratingExt(x))

revDF2=revDF[['cl_review','ratingPer']]

def rating2number(x):
    if x >= .5:
        return 1
    else:
        return 0
    
revDF2.ratingPer2=revDF2.ratingPer.apply(lambda x : rating2number(x))

revDF2['text'] = revDF2['cl_review'].apply(lambda x: x.lower())
revDF2['text'] = revDF2['cl_review'].apply((lambda x: re.sub('[^a-zA-z0-9\s.]','',x)))



C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:36: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
revDF2['text2']=revDF2.text.apply(lambda x : x.split('Rated'))

C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
revDF3=revDF2.copy()

In [13]:
revDF3.head()

,cl_review,ratingPer,text,text2
0,",Rated 4.0,,n A beautiful place to dine in.Th...",0.82,Rated 4.0n A beautiful place to dine in.The i...,"[, 4.0n A beautiful place to dine in.The int..."
1,",Rated 4.0,,n Had been here for dinner with f...",0.82,Rated 4.0n Had been here for dinner with fami...,"[, 4.0n Had been here for dinner with family..."
2,",Rated 3.0,,n Ambience is not that good enoug...",0.76,Rated 3.0n Ambience is not that good enough a...,"[, 3.0n Ambience is not that good enough and..."
3,",Rated 4.0,,n Great food and proper Karnataka...",0.74,Rated 4.0n Great food and proper Karnataka st...,"[, 4.0n Great food and proper Karnataka styl..."
4,",Rated 4.0,,n Very good restaurant in neighbo...",0.76,Rated 4.0n Very good restaurant in neighbourh...,"[, 4.0n Very good restaurant in neighbourhoo..."


In [14]:
revDF3=revDF3[['ratingPer','text2']]

In [15]:
revDF3.head()

,ratingPer,text2
0,0.82,"[, 4.0n A beautiful place to dine in.The int..."
1,0.82,"[, 4.0n Had been here for dinner with family..."
2,0.76,"[, 3.0n Ambience is not that good enough and..."
3,0.74,"[, 4.0n Great food and proper Karnataka styl..."
4,0.76,"[, 4.0n Very good restaurant in neighbourhoo..."


In [16]:
lst_col = 'text2'

revDF3.columns.difference([lst_col])

Index(['ratingPer'], dtype='object')

In [17]:
lst_col = 'text2'
#revDF4=pd.DataFrame({col:np.repeat(revDF3[col].values, revDF3[lst_col].str.len())
#              for col in revDF3.columns.difference([lst_col])
#             }).assign(**{lst_col:np.concatenate(revDF3[lst_col].values)})[revDF3.columns.tolist()]

In [18]:
#revDF4=pd.DataFrame({'ratingPer':np.repeat(revDF3.ratingPer.values, revDF3.text2.str.len()),
#                        'text2':np.concatenate(revDF3.text2.values)})

In [19]:
#df = pd.DataFrame({
#    "a" : [1,2,3],
#    "b" : [[' ','asd','ase'],['awe','dfgg','sdasda'],['wewe']],
#    "c" : [5,6,7]
#})

#df
#df.set_index('a').b.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index()

In [20]:
revDF4=revDF3.set_index('ratingPer').text2.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(str).reset_index()


In [21]:
revDF4.columns=['ratingPer','text2']

In [22]:
revDF4.head()

,ratingPer,text2
0,0.82,
1,0.82,4.0n A beautiful place to dine in.The interi...
2,0.82,4.0n I was here for dinner with my family on...
3,0.82,2.0n Its a restaurant near to Banashankari B...
4,0.82,4.0n We went here on a weekend and one of us...


In [23]:
def func1(x):
    if len(x) == 0:
        return np.nan
    else:
        return x


revDF4['text3']=revDF4['text2'].apply(lambda x: func1(x))
revDF4=revDF4.dropna()
revDF4=revDF4.reset_index()
revDF4=revDF4.drop(['index','text3'],axis=1)

In [24]:
def textNumberSeparator(x):
    return re.split('\.[0-9]+n\s+',x)
    

In [25]:
revDF4['text3']=revDF4['text2'].apply(lambda x :textNumberSeparator(x))

In [26]:
revDF4['rating2']=revDF4['text3'].apply(lambda x : x[0])

In [27]:
revDF4.head()

,ratingPer,text2,text3,rating2
0,0.82,4.0n A beautiful place to dine in.The interi...,"[ 4, A beautiful place to dine in.The interior...",4
1,0.82,4.0n I was here for dinner with my family on...,"[ 4, I was here for dinner with my family on a...",4
2,0.82,2.0n Its a restaurant near to Banashankari B...,"[ 2, Its a restaurant near to Banashankari BDA...",2
3,0.82,4.0n We went here on a weekend and one of us...,"[ 4, We went here on a weekend and one of us h...",4
4,0.82,5.0n The best thing about the place is its a...,"[ 5, The best thing about the place is its amb...",5


In [28]:
def ratingExt(x):
    if len(x) > 1:
        return x[1]
    else:
        np.nan    

revDF4['text4']=revDF4['text3'].apply(lambda x : ratingExt(x))

In [29]:
#re.split('\.[0-9]+n\s+',revDF4['text2'][2])

In [30]:
revDF4.head(),revDF4.shape

(   ratingPer                                              text2  \
 0       0.82   4.0n  A beautiful place to dine in.The interi...   
 1       0.82   4.0n  I was here for dinner with my family on...   
 2       0.82   2.0n  Its a restaurant near to Banashankari B...   
 3       0.82   4.0n  We went here on a weekend and one of us...   
 4       0.82   5.0n  The best thing about the place is its a...   
 
                                                text3 rating2  \
 0  [ 4, A beautiful place to dine in.The interior...       4   
 1  [ 4, I was here for dinner with my family on a...       4   
 2  [ 2, Its a restaurant near to Banashankari BDA...       2   
 3  [ 4, We went here on a weekend and one of us h...       4   
 4  [ 5, The best thing about the place is its amb...       5   
 
                                                text4  
 0  A beautiful place to dine in.The interiors tak...  
 1  I was here for dinner with my family on a week...  
 2  Its a restaurant near to B

In [31]:
#revDF4[revDF4['text4']==np.nan]
null_columns=revDF4.columns[revDF4.isnull().any()]
revDF4.isnull().sum()
#print(revDF4[revDF4["text4"].isnull()][null_columns])


ratingPer       0
text2           0
text3           0
rating2         0
text4        2583
dtype: int64

In [32]:
revDF4.iloc[2598,]['text3']

[' 5 mainly for the location and service. .nThank you. ']

In [33]:
 #revDF4=revDF3.set_index('ratingPer').text2.apply(pd.Series).stack().reset_index(level=-1, drop=True).astype(int).reset_index()

In [34]:
revDF4=revDF4.dropna()

In [35]:
revDF4['rating2']=revDF4['rating2'].astype(int)

In [36]:
revDF4.head()

,ratingPer,text2,text3,rating2,text4
0,0.82,4.0n A beautiful place to dine in.The interi...,"[ 4, A beautiful place to dine in.The interior...",4,A beautiful place to dine in.The interiors tak...
1,0.82,4.0n I was here for dinner with my family on...,"[ 4, I was here for dinner with my family on a...",4,I was here for dinner with my family on a week...
2,0.82,2.0n Its a restaurant near to Banashankari B...,"[ 2, Its a restaurant near to Banashankari BDA...",2,Its a restaurant near to Banashankari BDA. Me ...
3,0.82,4.0n We went here on a weekend and one of us...,"[ 4, We went here on a weekend and one of us h...",4,We went here on a weekend and one of us had th...
4,0.82,5.0n The best thing about the place is its a...,"[ 5, The best thing about the place is its amb...",5,The best thing about the place is its ambiance...


In [37]:
revDF4.describe()

,ratingPer,rating2
count,1.306875e+06,1.306875e+06
mean,7.731172e-01,3.605508e+00
std,1.128684e-01,1.297528e+00
min,3.600000e-01,1.000000e+00
25%,7.200000e-01,3.000000e+00
50%,7.800000e-01,4.000000e+00
75%,8.600000e-01,5.000000e+00
max,9.800000e-01,5.000000e+00


In [38]:
def rater(x):
    if x > 3:
        return 1
    else:
        return 0
revDF4['ratingNew']=revDF4['rating2'].apply(lambda x : rater(x))

In [39]:
revDF4.head()

,ratingPer,text2,text3,rating2,text4,ratingNew
0,0.82,4.0n A beautiful place to dine in.The interi...,"[ 4, A beautiful place to dine in.The interior...",4,A beautiful place to dine in.The interiors tak...,1
1,0.82,4.0n I was here for dinner with my family on...,"[ 4, I was here for dinner with my family on a...",4,I was here for dinner with my family on a week...,1
2,0.82,2.0n Its a restaurant near to Banashankari B...,"[ 2, Its a restaurant near to Banashankari BDA...",2,Its a restaurant near to Banashankari BDA. Me ...,0
3,0.82,4.0n We went here on a weekend and one of us...,"[ 4, We went here on a weekend and one of us h...",4,We went here on a weekend and one of us had th...,1
4,0.82,5.0n The best thing about the place is its a...,"[ 5, The best thing about the place is its amb...",5,The best thing about the place is its ambiance...,1


In [40]:
#t2=list(set(cln(revDF['reviews_list'][13]).split('RATEDn')))[-1].split('Rated ')[0]

In [41]:
# traintest split
X_train, X_test, Y_train, Y_test = train_test_split(revDF4['text4'],revDF4.ratingNew, test_size = 0.05, random_state = 42)

In [42]:
X_test.head()

523480     kundana...... we went for a movie to vegacity ...
1135722    A person can never be too old for CupcakesnnHe...
266784     Good food and drink s view is the main USP it ...
879908     bad service  the thing which you order you get...
169157     Great place with wonderful ambience and helpfu...
Name: text4, dtype: object

In [43]:
#max_fatures = 2000
#tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer = Tokenizer(nb_words=100, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,split=' ')

#tokenizer.fit_on_texts(revDF4['text4'].values)

tokenizer.fit_on_texts(X_train.values)

#X = tokenizer.texts_to_sequences(revDF4['text4'].values)
X = tokenizer.texts_to_sequences(X_train.values)

X = pad_sequences(X, maxlen=100,padding = 'post')



C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\keras_preprocessing\text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [44]:
X_train.values

array(['To be honest when no other place was delivering Shawarma plate to my address I had to forcefully order from here . I ordered without any expectations cos I havent heard much about this place. To my surprise the Shawarma plate was really good I havent tried any other items from here but Im planning to try a few soon ',
       'I got parcel Mutton puff  Chicken puff Black forest pastry and Pineapple pastry from here . All of them were fresh and flavourful. Pineapple pastry was yummy. ',
       'One of the few restaurants in Marathalli which serves Hookah I was really impressed with the ambiance as it has been set up with bean bags on one side of the restaurant and it also has a very big TV screen but was little irritating to us as it was so loud and after telling 3 to 4 times they reduced the volume.nnThe service section has to pick up pace as they were very slow in bringing the food we ordered for double apple cheesy macaroni chicken hotdog Crispy Paneer Burger Paneer Roll and c

In [45]:
from sklearn.model_selection import train_test_split
X_train2, X_valid, Y_train2, Y_valid = train_test_split(X,Y_train, test_size = 0.10, random_state = 42)

In [46]:
X_train2[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  7, 49,  5,  6,  3,  5,  5, 69, 35, 52, 11,  1, 64,  5,
       70, 16,  2,  6, 19, 59, 26, 62, 77, 10, 10, 26, 10, 26, 10])

In [47]:
# create the model
embedding_vecor_length = 64
model = Sequential()
model.add(Embedding(100, embedding_vecor_length, input_length=X_train2.shape[1],dropout = 0.2))
model.add(LSTM(100,dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train2, Y_train2, validation_data=(X_valid, Y_valid), epochs=1, batch_size=64)

C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  after removing the cwd from sys.path.
C:\Users\OMR\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, dropout=0.2, recurrent_dropout=0.2)`
  """


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 64)           6400      
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 72,501
Trainable params: 72,501
Non-trainable params: 0
_________________________________________________________________
None
Train on 1117377 samples, validate on 124154 samples
Epoch 1/1
  47808/1117377 [>.............................] - ETA: 1:04:33 - loss: 0.4967 - acc: 0.7618

KeyboardInterrupt: 

In [172]:
t2="a nice place to eat"
X2 = tokenizer.texts_to_sequences([t2])
X2 = pad_sequences(X2, maxlen=100)


In [175]:
model=keras.models.load_model('modelsaved.h5')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [176]:
model.predict(X2)

array([[0.79711956]], dtype=float32)

In [248]:
#model.save('modelsaved.h5')

In [257]:


Xtest = tokenizer.texts_to_sequences(X_test.values)
Xtest = pad_sequences(Xtest, maxlen=100)


In [259]:
Xtest.shape

(65344, 100)

In [266]:
pred_test=model.predict_classes(Xtest)

In [267]:
pred_test.shape

(65344, 1)

In [273]:
#pred_test[0]

In [274]:
#Y_test.values[0]

In [272]:
f1_score(Y_test.values, pred_test, average='macro') 

0.8195998631683123

In [276]:
target_names = ['class 0', 'class 1']
print(classification_report(Y_test.values, pred_test, target_names=target_names))

             precision    recall  f1-score   support

    class 0       0.82      0.71      0.76     23928
    class 1       0.85      0.91      0.88     41416

avg / total       0.84      0.84      0.83     65344



In [279]:
confusion_matrix(Y_test.values, pred_test)

array([[17054,  6874],
       [ 3740, 37676]], dtype=int64)

In [280]:
accuracy_score(Y_test.values, pred_test)

0.8375673359451518